# Score: 65.071% (performs a bit worse)

## I think the model was overfitting in the last submission

- Based on my basic data analysis it seems like some factors like embarkment may not matter

- So I'm going to test only factors that certainly matter like age and gender

In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../data/titanic/train.csv')

In [2]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
predictor_cols = ['Age', 'Sex']

y = data.Survived
X = data[predictor_cols]
X = pd.get_dummies(X)
train_x, test_x, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

imputer = Imputer()
train_x = imputer.fit_transform(train_x)
test_x = imputer.transform(test_x)

In [6]:
model = XGBRegressor()
model.fit(train_x, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [7]:
predictions = model.predict(test_x)

print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 0.2984988295176639


In [9]:
test = pd.read_csv('../data/titanic/test.csv')
test_x = test[predictor_cols]
test_x = pd.get_dummies(test_x)

In [10]:
submission = model.predict(test_x.as_matrix())

for i in range(len(submission)):
    result = submission[i]
    if result < 0.5:
        submission[i] = 0
    else:
        submission[i] = 1
submission = submission.astype(int)

In [12]:
import os
os.sys.path.append("/Users/pbezuhov/git/custom_libraries/")
import main

columns = ["PassengerId", "Survived"]
ids = test['PassengerId'].as_matrix()
data = list(zip(ids, list(submission)))
main.save_file('/Users/pbezuhov/git/Kaggle/submissions/titanic/3_removing_var.csv', data, columns)